In [1]:
import dateutil.parser
import glob,os

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.cluster import KMeansClusterer

import numpy as np
import pandas as pd
import operator

from gensim.models import Doc2Vec, Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier

C:\Users\ab356917\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
path = 'C:\\Users\\ab356917\\Downloads\\Cluster-data'
data = []
for infile in glob.glob( os.path.join(path, '*.*') ):
       with open(infile) as file:
            string = file.readlines()
            for each_line in string:
                if len(each_line.split())>5: # to remove time and name from the doc
                    data.extend(each_line.split(":")[2:])

In [3]:
# sentenceLabeled = []
# for sentenceID, sentence in enumerate(data):
#     sentenceL = TaggedDocument(words=sentence.split(), tags = ['SENT_%s' %sentenceID])
#     sentenceLabeled.append(sentenceL)

In [4]:
# doc2vec_model = Doc2Vec(vector_size=300, window=10, min_count=0, workers=11, alpha=0.005, min_alpha=0.025)
# doc2vec_model.build_vocab(sentenceLabeled)

In [5]:
# for epoch in range(20):
#     print(epoch ,"is running")
#     doc2vec_model.train(sentenceLabeled,total_examples=len(sent_tokenize(data)),epochs=2000)
#     doc2vec_model.alpha -= 0.0008  # decrease the learning rate
#     doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
#     doc2vec_model.save("doc2vec_model-withoutTime-g.model")
# textVect = doc2vec_model.docvecs.doctag_syn0

In [6]:
doc2vec_model = Doc2Vec.load("doc2vec_model-withoutTime-g.model")
textVect = doc2vec_model.docvecs.vectors_docs

In [7]:
## K-means ##
num_clusters = 3
km = KMeans(n_clusters=num_clusters)
km.fit(textVect)
clusters = km.labels_.tolist()

## Print Sentence Clusters ##
cluster_info = {'sentence': data, 'cluster' : clusters}
sentenceDF = pd.DataFrame(cluster_info, index=[clusters], columns = ['sentence','cluster'])

for num in range(num_clusters):
    print()
    print("Sentence cluster %d: " %int(num+1), end='')
    print()
    for sentence in sentenceDF.ix[num]['sentence'].values.tolist():
        print(' %s ' %sentence, end='')
    print()


Sentence cluster 1: 
  Welcome to HP Customer Support for Notebook Support. My name is Akash and I will be your technical support specialist today.
   Not good my new computer is not working
   Do not worry, I will do my best to help you./
   If I am correct, after logging in the icons take time to appear.
   Are you chatting from the same notebook?
   It was recently updated too.  No my phone 
   May I know if the notebook is powered on?
   Yeah its on. Im running a system check
   Its at 6% it says it will take 5 hours
   And restart the notebook.
   It is at the page where i enter my password
   Now please press CTRL + ALT + DEL key simultaneously.
   Screen is black
   This will open the task bar.
   Once the task bar open then you will get an option as "File".
   I dont see file. I see lock,switch user,sign out,change password and task mangager
   Then please select "Task Manager"".
   Then you will get an option as "File" on the top.
   Ok so im in run new task
   A new window w

C:\Users\ab356917\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  from ipykernel import kernelapp as app


In [8]:
# Importing files containing Ques, Ans and Others data
ques_file = open('C:\\Users\\ab356917\\Downloads\\ques.txt','r').readlines()
ans_file = open('C:\\Users\\ab356917\\Downloads\\ans.txt','r').readlines()
others_file = open('C:\\Users\\ab356917\\Downloads\\others.txt','r').readlines()

In [9]:
# All Corpus Words including Questions, Answers and Others
data_AllWords = []
for i in data:
    data_AllWords.extend(word_tokenize(i.lower()))

In [10]:
# List of all Concatenated Questions, Concatenated Answers, Concatenated Others
Questions_All = [" ".join([i.replace('\n','') for i in ques_file])]
Answers_All = [" ".join([i.replace('\n','') for i in ans_file])]
Others_All = [" ".join([i.replace('\n','') for i in others_file])]

# List of ques, ans and others
ques_list = [i.replace('\n','') for i in ques_file]
ans_list = [i.replace('\n','') for i in ans_file]
others_list = [i.replace('\n','') for i in others_file]

# List containing All concatenated Questions, list of Answers and List of Others
AllQuestion_ans_others = Questions_All + ans_list + others_list
AllAnswer_ques_others = Answers_All + ques_list + others_list
AllOthers_ans_ques = Others_All + ans_list + ques_list

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [12]:
stop_words = ['i','me', 'my', 'myself','we','our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their','theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being','have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing','a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for','with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'all','any', 'both', 'each', 'few','more', 'most', 'other','some','such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will','just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn','weren','won','wouldn']

# Questions features

In [13]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(AllQuestion_ans_others).todense() #fit the vectorizer to synopses

In [14]:
ques_features = list(zip(tfidf_vectorizer.get_feature_names(), tfidf_matrix.tolist()[0]))
ques_features = sorted(ques_features, key=lambda x: x[1], reverse= True)
ques_features = [i for i in ques_features if i[1]>0]

# Answers features

In [15]:
#define vectorizer parameters
tfidf_vectorizer_ans = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix_ans = tfidf_vectorizer_ans.fit_transform(AllAnswer_ques_others).todense() #fit the vectorizer to synopses

In [16]:
ans_features = list(zip(tfidf_vectorizer_ans.get_feature_names(), tfidf_matrix_ans.tolist()[0]))
ans_features = sorted(ans_features, key=lambda x: x[1], reverse= True)
ans_features = [i for i in ans_features if i[1]>0]

# Others feature

In [17]:
#define vectorizer parameters
tfidf_vectorizer_others = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix_others = tfidf_vectorizer_others.fit_transform(AllOthers_ans_ques).todense() #fit the vectorizer to synopses

In [18]:
others_features = list(zip(tfidf_vectorizer_others.get_feature_names(), tfidf_matrix_others.tolist()[0]))
others_features = sorted(others_features, key=lambda x: x[1], reverse= True)
others_features = [i for i in others_features if i[1]>0]

In [19]:
all_features = [i[0] for i in ques_features] + [i[0] for i in ans_features] + [i[0] for i in others_features]

In [20]:
all_features = list(set(all_features))

In [21]:
all_features = [x for x in all_features if not x.strip().isdigit() ]

# Frequency Matrix

In [22]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(all_features)
#print("Vocabulary:", count_vectorizer.vocabulary_)

<435x435 sparse matrix of type '<class 'numpy.int64'>'
	with 435 stored elements in Compressed Sparse Row format>

In [23]:
freq_term_matrix = count_vectorizer.transform(ques_list + ans_list + others_list)
print(freq_term_matrix.todense()) 

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [24]:
rf = RandomForestClassifier(n_estimators=100)

In [25]:
class_list = ["Ques"]*len(ques_list)+["Ans"]*len(ans_list)+["Others"]*len(others_list)

In [26]:
rf.fit(freq_term_matrix.todense(),class_list)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
#results = rf.predict(freq_term_matrix.todense())

In [28]:
class_of = count_vectorizer.transform(["how are you?"])

In [29]:
ordered_features = [key for key, value in sorted(count_vectorizer.vocabulary_.items(), key=operator.itemgetter(1))]

In [30]:
rf.predict(class_of)

array(['Ans'], 
      dtype='<U6')

In [31]:
def get_class_features(list_of_sentences, count_vectorizer = count_vectorizer, ordered_features = ordered_features, rf = rf ):
    
    class_of = count_vectorizer.transform(list_of_sentences)
    class_is = rf.predict(class_of)
    
    features = []
    for each_class in class_of.todense().tolist():
        test = list(zip(ordered_features, each_class))
        features.append([i for i in test if i[1]>0])
    
    class_feature = {}
    for index, each_sentence in enumerate(list_of_sentences):
        class_feature.update({each_sentence:{"class": class_is[index], "feature": features[index]}})
    return class_feature

In [32]:
input_document = ["Do i ask check the create this task with administrative privileges?", "My laptop is not working.","How are you?","While troubleshoot I checked the device drivers and software and let me tell you, your HP laptop is just fit and dandy."]

In [33]:
test = list(zip(ordered_features, class_of.todense().tolist()[0]))

In [34]:
get_class_features(input_document)

{'Do i ask check the create this task with administrative privileges?': {'class': 'Ques',
  'feature': [('administrative', 1),
   ('ask', 1),
   ('check', 1),
   ('create', 1),
   ('privileges', 1),
   ('task', 1)]},
 'How are you?': {'class': 'Ans', 'feature': [('how', 1)]},
 'My laptop is not working.': {'class': 'Ques',
  'feature': [('laptop', 1), ('working', 1)]},
 'While troubleshoot I checked the device drivers and software and let me tell you, your HP laptop is just fit and dandy.': {'class': 'Ans',
  'feature': [('checked', 1),
   ('dandy', 1),
   ('device', 1),
   ('drivers', 1),
   ('fit', 1),
   ('hp', 1),
   ('laptop', 1),
   ('let', 1),
   ('software', 1),
   ('tell', 1),
   ('troubleshoot', 1)]}}

## train test k fold

In [35]:
from sklearn.metrics import accuracy_score

In [37]:
print("Train Accuracy :: ", accuracy_score(['Ques']*len(ques_file), rf.predict(count_vectorizer.transform(ques_list))))

Train Accuracy ::  0.992805755396


In [38]:
print("Train Accuracy :: ", accuracy_score(['Ans']*len(ans_file), rf.predict(count_vectorizer.transform(ans_list))))

Train Accuracy ::  0.652631578947
